# Лабораторная работа №3
## Выполнил студент группы БВТ2002 Айвазян Арина

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [1]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

### Ввод с клавиатуры

In [3]:
main_string = 'Our BIG string which contains this word and also that word (big)'
string_to_find = 'big'
is_case_sensitive = False

print('enter main string or leave empty for default string')
buffer = input()
if buffer != '':
    main_string = buffer
    buffer = ''
print('enter string to find or leave empty for default string')
buffer = input()
if buffer != '':
    string_to_find = buffer
    buffer = ''

print('should a search be case sensitive? Yes:(input "Y") No:(input anything else)')
buffer = input()
if buffer == 'Y':
    is_case_sensitive = True
    
    
# Our BIG string which contains this word and also that word (big)
# big
# Y

enter main string or leave empty for default string
 Our BIG string which contains this word and also that word (big)
enter string to find or leave empty for default string
 big
should a search be case sensitive? Yes:(input "Y") No:(input anything else)
 Y


#### Алгоритм Кнута-Морриса-Пратта

In [8]:
def prefix(s):
    str_prefix = [0]*len(s)
    for i in range(1, len(s)):
        k = str_prefix[i-1]
        while k > 0 and s[k] != s[i]:
            k = str_prefix[k-1]
        if s[k] == s[i]:
            k = k + 1
        str_prefix[i] = k
    return str_prefix


def kmp(s_main, s_sought, case_sensitive=True):
    if not case_sensitive:
        s_main = s_main.lower()
        s_sought = s_sought.lower()

    index = -1
    str_prefix = prefix(s_main)
    k = 0
    for i in range(len(s_sought)):
        while k > 0 and s_main[k] != s_sought[i]:
            k = str_prefix[k-1]
        if s_main[k] == s_sought[i]:
            k = k + 1
        if k == len(s_main):
            index = i - len(s_main) + 1
            break
    return index

print(kmp(string_to_find, main_string, is_case_sensitive))
print(kmp(string_to_find, main_string, False))

4
4


#### Упрощенный алгоритм Бойера-Мура

In [6]:
def build_table(s):
    dic_table = dict()
    length = len(s)
    for i in range(length - 1):
        dic_table[s[i]] = length - i - 1
    if not s[length - 1] in dic_table:
        dic_table[s[length - 1]] = length
    return dic_table


def boyer_mur_search(s_main, s_sought, case_sensitive=True):
    if not case_sensitive:
        s_main = s_main.lower()
        s_sought = s_sought.lower()

    dic_table = build_table(s_sought)
    s_sought_len = len(s_sought)
    current_step = s_sought_len - 1

    while current_step < len(s_main):  # основной цикл для поиска подстроки

        s_sought_i = s_sought_len - 1  # индекс для подстроки
        for i in range(current_step, current_step - s_sought_len, -1):
            if s_main[i] != s_sought[s_sought_i]:  # слово не совпало. Указываем необходимый шаг и выходим из for
                if s_main[i] in dic_table:
                    step = dic_table[s_main[i]]
                else:
                    step = s_sought_len
                break

            if i == current_step - s_sought_len + 1:  # последняя итерация for. Все символы совпали, возвращаем индекс
                return i
            s_sought_i -= 1

        current_step += step  # двигаем место поиска на шаг
        
print(boyer_mur_search(main_string, string_to_find, case_sensitive=is_case_sensitive))
print(boyer_mur_search(main_string, string_to_find, case_sensitive=False))

4
4


### Сравнение скорости работы

In [7]:
import timeit

print("Поиск Бойера-Мура          " + str(timeit.timeit("boyer_mur_search(main_string, string_to_find, case_sensitive=is_case_sensitive)", number=1, globals=globals())))
print("Поиск Кнута-Морриса-Пратта " + str(timeit.timeit("kmp(string_to_find, main_string, is_case_sensitive)", number=1, globals=globals())))
print("Поиск по умолчанию         " + str(timeit.timeit("main_string.find(string_to_find)", number=1, globals=globals())))

Поиск Бойера-Мура          1.250000013897079e-05
Поиск Кнута-Морриса-Пратта 1.990000009755022e-05
Поиск по умолчанию         1.99999976757681e-06


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [9]:
def print_pyatnashki(array):
    """
    Вывод пятнашек в красивом виде
    :param array:
    :return:
    """
    int_separate = 0
    for x in array:
        if x <= 9:
            print(' ' + str(x), end=" ")
        else:
            print(x, end=" ")
        int_separate += 1
        if int_separate >= 4:
            int_separate = 0
            print()


def solvable(array):
    """
    Формуля для определения решаемости пятнашек взята из википедии:
    https://ru.wikipedia.org/wiki/%D0%98%D0%B3%D1%80%D0%B0_%D0%B2_15#%D0%9C%D0%B0%D1%82%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%BE%D0%BF%D0%B8%D1%81%D0%B0%D0%BD%D0%B8%D0%B5
    :param array: наш массив для пятнашек
    :return: bool обозначающий, можно ли решить данные пятнашки
    """
    N = 0
    for x in array:
        if x == 0:
            N += array.index(0) // 4 + 1
        else:
            k = 0
            for i in range(array.index(x), len(array)):
                if array[i] < x and array[i] != 0:
                    k += 1
            N += k
    if N % 2 == 0:
        print(N)
        return True
    return False


arr_moves = []


def swap(array, empty_i, swap_this_i):
    """
    Функция для передвижения клеток
    :param array: наш массив для пятнашек
    :param empty_i: индекс пустой клетки
    :param swap_this_i: индекс клетки, которую мы собираемся двигать
    :return: индекс, на котором теперь находится пустая клетка
    """
    arr_moves.append(array[swap_this_i])
    array[empty_i], array[swap_this_i] = array[swap_this_i], array[empty_i]
    return swap_this_i


def rightmost_row_sort(array, empty_i):
    """
    Специальная последовательность перемещений, чтобы отсортировать цифры самого правого столбца (4,8)
    :param array: наш массив для пятнашек
    :param empty_i: индекс пустой клетки
    :return: индекс, на котором теперь находится пустая клетка
    """
    empty_i = swap(array, empty_i, empty_i - 1)
    empty_i = swap(array, empty_i, empty_i - 4)
    empty_i = swap(array, empty_i, empty_i - 1)
    empty_i = swap(array, empty_i, empty_i - 4)
    empty_i = swap(array, empty_i, empty_i + 1)
    empty_i = swap(array, empty_i, empty_i + 1)
    empty_i = swap(array, empty_i, empty_i + 4)
    empty_i = swap(array, empty_i, empty_i - 1)
    empty_i = swap(array, empty_i, empty_i - 4)
    empty_i = swap(array, empty_i, empty_i - 1)
    empty_i = swap(array, empty_i, empty_i + 4)


    return empty_i



def best_choice(best_array, origin, def_choice, choice2):
    index_origin = best_array.index(origin)
    for index in range(index_origin, 16):
        if index >= len(best_array):
            index = 0
        if best_array[index] == def_choice:
            return def_choice
        if best_array[index] == choice2:
            return choice2
    return def_choice



def pyatnashki(array):
    if not solvable(array):
        return False

    arr_moves.clear()
    empty_i = -1
    for cur_numb in range(1, 9):

        cur_numb_i = -1
        for i in range(16):  # находим индекс рассматриваемой цифры
            if array[i] == cur_numb:
                cur_numb_i = i

        empty_i = array.index(0)
        desirable_i = cur_numb - 1

        # [1] Помещаем пустую клетку рядом с клеткой числа для перемещения
        row_of_our_number = (cur_numb_i // 4) + 1
        while abs(empty_i - cur_numb_i) != 4 and (abs(empty_i - cur_numb_i) != 1 or empty_i // 4 != cur_numb_i // 4):
            if empty_i // 4 + 1 < row_of_our_number:
                empty_i = swap(array, empty_i, empty_i + 4)
            elif cur_numb_i % 4 > empty_i % 4:
                empty_i = swap(array, empty_i, empty_i + 1)
            elif empty_i // 4 + 1 > row_of_our_number:
                empty_i = swap(array, empty_i, empty_i - 4)

            else:
                empty_i = swap(array, empty_i, empty_i - 1)

        # [2] Определяем куда нужно переместить наше значение и двигаем пустую клетку соответственно
        while cur_numb_i != desirable_i:

            if cur_numb_i % 4 > desirable_i % 4:
                if cur_numb_i // 4 + 1 == 4:  # находится на 4 ряду
                    if empty_i - cur_numb_i == 1:
                        empty_i = swap(array, empty_i, empty_i - 4)
                        empty_i = swap(array, empty_i, empty_i - 1)
                    if cur_numb_i - empty_i == 4:
                        empty_i = swap(array, empty_i, empty_i - 1)
                        empty_i = swap(array, empty_i, empty_i + 4)
                    if cur_numb_i - empty_i == 1:
                        # один шаг для клетки, тысяча строчек кода для человечества
                        empty_i = swap(array, empty_i, empty_i + 1)
                        cur_numb_i -= 1
                else:

                    if cur_numb_i - empty_i == 4:
                        if cur_numb_i % 4 == 3:
                            empty_i = swap(array, empty_i, empty_i - 1)
                            empty_i = swap(array, empty_i, empty_i + 4)
                        else:
                            empty_i = swap(array, empty_i, empty_i + 1)
                            empty_i = swap(array, empty_i, empty_i + 4)
                    if empty_i - cur_numb_i == 1:
                        empty_i = swap(array, empty_i, empty_i + 4)
                        empty_i = swap(array, empty_i, empty_i - 1)
                    if empty_i - cur_numb_i == 4:
                        empty_i = swap(array, empty_i, empty_i - 1)
                        empty_i = swap(array, empty_i, empty_i - 4)
                    if cur_numb_i - empty_i == 1:
                        # один шаг для клетки, тысяча строчек кода для человечества
                        empty_i = swap(array, empty_i, empty_i + 1)
                        cur_numb_i -= 1

            elif cur_numb_i % 4 < desirable_i % 4:
                if cur_numb_i // 4 + 1 == 4:  # находится на 4 ряду
                    if cur_numb_i - empty_i == 1:
                        empty_i = swap(array, empty_i, empty_i - 4)
                        empty_i = swap(array, empty_i, empty_i + 1)
                    if cur_numb_i - empty_i == 4:
                        empty_i = swap(array, empty_i, empty_i + 1)
                        empty_i = swap(array, empty_i, empty_i + 4)
                    if empty_i - cur_numb_i == 1:
                        # один шаг для клетки, тысяча строчек кода для человечества
                        empty_i = swap(array, empty_i, empty_i - 1)
                        cur_numb_i += 1
                else:
                    if cur_numb_i - empty_i == 4:
                        empty_i = swap(array, empty_i, empty_i + 1)
                        empty_i = swap(array, empty_i, empty_i + 4)
                    if cur_numb_i - empty_i == 1:
                        empty_i = swap(array, empty_i, empty_i + 4)
                        empty_i = swap(array, empty_i, empty_i + 1)
                    if empty_i - cur_numb_i == 4:
                        empty_i = swap(array, empty_i, empty_i + 1)
                        empty_i = swap(array, empty_i, empty_i - 4)
                    if empty_i - cur_numb_i == 1:
                        # один шаг для клетки, тысяча строчек кода для человечества
                        empty_i = swap(array, empty_i, empty_i - 1)
                        cur_numb_i += 1

            else:
                if cur_numb_i % 4 == 3:  # Находится на 4 столбце
                    if cur_numb_i // 4 > desirable_i // 4 + 1:
                        if empty_i - cur_numb_i == 4:
                            empty_i = swap(array, empty_i, empty_i - 1)
                            empty_i = swap(array, empty_i, empty_i - 4)
                        if cur_numb_i - empty_i == 1:
                            empty_i = swap(array, empty_i, empty_i - 4)
                            empty_i = swap(array, empty_i, empty_i + 1)
                        if cur_numb_i - empty_i == 4:
                            # один шаг для клетки, тысяча строчек кода для человечества
                            empty_i = swap(array, empty_i, empty_i + 4)
                            cur_numb_i -= 4
                    else:
                        if cur_numb_i - empty_i == 4:
                            empty_i = swap(array, empty_i, empty_i + 4)
                            cur_numb_i = desirable_i
                            continue
                        if cur_numb_i - empty_i == 1:
                            empty_i = swap(array, empty_i, empty_i + 4)
                            empty_i = swap(array, empty_i, empty_i + 1)
                        if empty_i - cur_numb_i == 4:
                            # Финальный этап сортировки для чисел 4 и 8
                            empty_i = rightmost_row_sort(array, empty_i)
                            cur_numb_i = desirable_i
                else:
                    if cur_numb_i // 4 > desirable_i // 4:
                        if cur_numb_i // 4 + 1 == 4:  # находится на 4 ряду
                            if cur_numb_i - empty_i == 1:
                                empty_i = swap(array, empty_i, empty_i - 4)
                                empty_i = swap(array, empty_i, empty_i + 1)
                            if empty_i - cur_numb_i == 1:
                                empty_i = swap(array, empty_i, empty_i - 4)
                                empty_i = swap(array, empty_i, empty_i - 1)
                            if cur_numb_i - empty_i == 4:
                                # один шаг для клетки, тысяча строчек кода для человечества
                                empty_i = swap(array, empty_i, empty_i + 4)
                                cur_numb_i -= 4
                        else:
                            if cur_numb_i - empty_i == 1:
                                empty_i = swap(array, empty_i, empty_i + 4)
                                empty_i = swap(array, empty_i, empty_i + 1)
                            if empty_i - cur_numb_i == 4:
                                empty_i = swap(array, empty_i, empty_i + 1)
                                empty_i = swap(array, empty_i, empty_i - 4)
                            if empty_i - cur_numb_i == 1:
                                empty_i = swap(array, empty_i, empty_i - 4)
                                empty_i = swap(array, empty_i, empty_i - 1)
                            if cur_numb_i - empty_i == 4:
                                # один шаг для клетки, тысяча строчек кода для человечества
                                empty_i = swap(array, empty_i, empty_i + 4)
                                cur_numb_i -= 4
    # [3] 3 и 4 строки
    sought_arr = [9, 10, 11, 12, 15, 14, 13]  # массив, который мы желаем в итоге получить из двух строк
    while True:
        if empty_i == 8:
            empty_i = swap(array, empty_i, empty_i + 1)
        elif empty_i == 9:
            index = array.index(best_choice(sought_arr, array[8], array[10], array[13]))
            if index == 13 and sought_arr.index(array[8]) > sought_arr.index(array[14]):
                empty_i = swap(array, empty_i, index)
                empty_i = swap(array, empty_i, empty_i - 1)
            else:
                empty_i = swap(array, empty_i, index)

        elif empty_i == 10:
            index = array.index(best_choice(sought_arr, array[9], array[11], array[14]))
            if index == 14 and sought_arr.index(array[9]) > sought_arr.index(array[15]):
                empty_i = swap(array, empty_i, index)
                empty_i = swap(array, empty_i, empty_i - 1)
                empty_i = swap(array, empty_i, empty_i - 1)
                empty_i = swap(array, empty_i, empty_i - 4)
                empty_i = swap(array, empty_i, empty_i + 1)
                empty_i = swap(array, empty_i, empty_i + 1)

            else:
                empty_i = swap(array, empty_i, index)

        elif empty_i == 11:
            empty_i = swap(array, empty_i, empty_i + 4)
        elif empty_i == 12:
            empty_i = swap(array, empty_i, empty_i - 4)
        elif empty_i == 13:
            index = array.index(best_choice(sought_arr, array[14], array[12], array[9]))
            if index == 9 and sought_arr.index(array[14]) > sought_arr.index(array[8]):
                empty_i = swap(array, empty_i, index)
                empty_i = swap(array, empty_i, empty_i + 1)
                empty_i = swap(array, empty_i, empty_i + 1)
                empty_i = swap(array, empty_i, empty_i + 4)
                empty_i = swap(array, empty_i, empty_i - 1)
                empty_i = swap(array, empty_i, empty_i - 1)
            else:
                empty_i = swap(array, empty_i, index)

        elif empty_i == 14:
            index = array.index(best_choice(sought_arr, array[15], array[13], array[10]))
            if index == 10 and sought_arr.index(array[15]) > sought_arr.index(array[9]):
                empty_i = swap(array, empty_i, index)
                empty_i = swap(array, empty_i, empty_i + 1)
                empty_i = swap(array, empty_i, empty_i - 4)
                empty_i = swap(array, empty_i, empty_i - 1)
            else:
                empty_i = swap(array, empty_i, index)

        else:
            if array[8] == 9 and array[9] == 10 and array[10] == 11 and array[11] == 12 and array[12] == 13 and array[13] == 14 and  array[14] == 15 and array[15] == 0:
                print_pyatnashki(array)
                return arr_moves
            empty_i = swap(array, empty_i, empty_i - 1)
            

pyatnashki_array = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 14, 0]
arr1 = [10, 6, 4, 13, 3, 5, 14, 1, 12, 8, 15, 0, 7, 11, 2, 9]
arr2 = [15, 1, 11, 7, 10, 4, 12, 14, 2, 8, 5, 13, 0, 6, 3, 9]
arr3 = [8, 15, 10, 5, 6, 9, 13, 4, 2, 1, 12, 7, 3, 0, 11, 14]
arr4 = [12, 2, 7, 3, 11, 6, 15, 4, 10, 5, 1, 9, 14, 13, 8, 0]

print(pyatnashki(arr1))
print(pyatnashki(arr2))
print(pyatnashki(arr4))
print(pyatnashki(arr4))
print(pyatnashki(pyatnashki_array))

54
 1  2  3  4 
 5  6  7  8 
 9 10 11 12 
13 14 15  0 
[15, 14, 1, 15, 14, 8, 5, 1, 8, 5, 12, 3, 1, 6, 10, 1, 3, 7, 11, 2, 5, 12, 2, 5, 12, 8, 6, 2, 8, 6, 4, 10, 2, 3, 7, 8, 6, 4, 3, 6, 4, 14, 15, 13, 10, 3, 14, 4, 5, 12, 9, 15, 4, 14, 13, 4, 14, 13, 6, 2, 3, 10, 4, 6, 10, 3, 2, 10, 13, 9, 12, 11, 8, 5, 10, 7, 5, 10, 9, 14, 14, 13, 6, 14, 13, 9, 7, 6, 9, 7, 11, 12, 15, 13, 14, 9, 7, 15, 12, 8, 10, 11, 15, 12, 8, 15, 12, 14, 13, 8, 14, 13, 8, 14, 13, 12, 6, 7, 9, 8, 12, 9, 7, 6, 15, 10, 11, 15, 9, 12, 14, 13, 10, 11, 15, 9, 11, 11, 12, 14, 13, 10, 11, 12, 14, 13, 10, 11, 12, 15, 9, 14, 13, 10, 11, 12, 15, 13, 13, 9, 14, 13, 9, 14, 13, 9, 10, 11, 12]
62
 1  2  3  4 
 5  6  7  8 
 9 10 11 12 
13 14 15  0 
[6, 8, 4, 10, 15, 1, 10, 4, 2, 6, 8, 3, 5, 12, 4, 2, 12, 4, 11, 10, 2, 12, 4, 5, 3, 4, 5, 3, 9, 13, 14, 11, 3, 14, 11, 7, 10, 3, 14, 9, 4, 5, 9, 11, 13, 4, 11, 13, 4, 11, 13, 14, 7, 4, 14, 7, 12, 2, 3, 10, 4, 12, 10, 3, 2, 9, 6, 8, 5, 6, 8, 5, 6, 8, 9, 15, 5, 9, 8, 6, 9, 8, 6, 13, 7, 10,

### Вывод

Мной были реализованы методы поиска подстроки в строке, и оценено время работы каждого алгоритма поиска на языке программирования Python. Так же была выполнена задача на решение "пятнашек"